# simple t-test

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pylab as plt
import matplotlib
from IPython.display import display, HTML, Image

import numpy as np
import pandas as pd
from sklearn import preprocessing
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

%matplotlib inline

In [3]:
from linker.models import Analysis
from linker.views.functions import get_last_analysis_data, get_groups, get_dataframes, get_standardized_df, \
    get_group_members
from linker.views.pipelines import WebOmicsInference
from linker.constants import *


        MATCH (n:Species) RETURN n.displayName AS name order by name        
        

            MATCH (tp:TopLevelPathway)-[:hasEvent*]->(p:Pathway)-[:hasEvent*]->(rle:ReactionLikeEvent)
            WHERE
                tp.displayName = 'Metabolism' AND
                tp.speciesName IN {species_list} AND
                (p)-[:hasEvent]->(rle)
            RETURN DISTINCT
                p.speciesName AS species_name,            
                p.displayName AS pathway_name,
                p.stId AS pathway_id                       
            ORDER BY species_name, pathway_name
        


### Load test data

In [4]:
analysis_id = 25

In [5]:
data_type = METABOLOMICS
analysis = Analysis.objects.get(pk=analysis_id)
analysis_data = get_last_analysis_data(analysis, data_type)
groups = get_groups(analysis_data)

In [6]:
data_df, design_df = get_dataframes(analysis_data, PKS)

In [7]:
data_df

,obs,compound_id,UN_1,UN4,INFEC_1,INFEC_2,INFEC_3,INFEC_4,HK1,HK2,HK3,HK4,padj_INFEC_vs_UN,FC_INFEC_vs_UN
compound_pk,,,,,,,,,,,,,,
C00001,False,H2o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00002,False,Atp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00003,False,Nad+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00004,False,Nadh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00005,False,Nadph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C14767,False,9(s)-hode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C15519,False,25-hydroxycholesterol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C15613,False,"(25r)-3alpha,7alpha,12alpha-trihydroxy-5beta-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Run t-test

In [8]:
case = 'HK'
control = 'UN'

In [9]:
wi = WebOmicsInference(data_df, design_df, data_type, min_value=5000)
result_df = wi.run_ttest(case, control)
result_df.head()

,padj,log2FoldChange
C00009,0.937964,0.031410
C00020,0.552928,-0.319297
C00025,0.552928,0.162212
C00037,0.552928,-0.380872
C00041,0.552928,-0.201749


### Run pathway analysis

In [14]:
data_df[data_df['obs'] == True]

,obs,compound_id,UN_1,UN4,INFEC_1,INFEC_2,INFEC_3,INFEC_4,HK1,HK2,HK3,HK4,padj_INFEC_vs_UN,FC_INFEC_vs_UN
compound_pk,,,,,,,,,,,,,,
C00009,True,Orthophosphate,3.814018e+05,3.128377e+05,4.795785e+05,2.300056e+05,2.320829e+05,4.491308e+05,2.366074e+05,5.360295e+05,3.217350e+05,3.806362e+05,0.871045,-0.077060
C00020,True,Amp,8.374511e+04,1.356138e+05,6.654536e+04,1.865876e+05,1.294665e+05,1.281130e+05,8.744820e+04,7.811514e+04,7.952553e+04,9.796303e+04,0.800010,0.168774
C00025,True,L-glutamate,2.097754e+07,1.879265e+07,2.175636e+07,1.712361e+07,3.783994e+07,1.914746e+07,2.641642e+07,2.008154e+07,2.286302e+07,2.009107e+07,0.778123,0.199115
C00037,True,Glycine,1.340802e+06,1.048188e+06,7.590960e+05,8.542596e+05,1.467938e+06,6.626538e+05,1.084838e+06,7.663168e+05,9.949456e+05,8.306521e+05,0.775449,-0.411692
C00041,True,L-alanine,3.537903e+06,2.808830e+06,4.275319e+06,3.713087e+06,7.178702e+06,3.782108e+06,3.134556e+06,2.539420e+06,2.901518e+06,2.443864e+06,0.775449,0.531459
C00051,True,Glutathione,6.568482e+05,8.306092e+05,8.893551e+05,8.226758e+05,9.625276e+05,1.227659e+06,1.198968e+06,1.017580e+06,1.532543e+06,1.158258e+06,0.775449,0.384573
C00064,True,L-glutamine,3.950205e+08,2.893683e+08,2.577768e+08,1.618516e+08,4.030473e+08,1.826548e+08,2.857788e+08,3.157292e+08,3.144469e+08,2.785741e+08,0.775449,-0.522202
C00077,True,L-ornithine,1.861422e+06,2.249538e+06,2.388442e+06,1.812233e+06,5.175261e+06,1.805338e+06,2.832114e+06,1.609296e+06,2.083633e+06,1.593964e+06,0.778123,0.301418
C00078,True,L-tryptophan,2.153219e+06,1.682301e+06,1.391535e+06,1.083955e+06,2.724574e+06,9.182302e+05,1.948601e+06,1.545008e+06,1.847820e+06,1.635050e+06,0.775449,-0.449480
